In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


In [2]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [3]:
at = pd.read_csv(path+'active_totes_20201210.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-0912.csv',names = ["timestamp","Pick Station","Availability","Blue Tote Loss","Grey Tote Loss"])
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'Faults20_11-10_12.csv')

In [4]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2230917628112) was called 1 times


2021-01-04 12:57:38,371 - NumExpr defaulting to 8 threads.


In [5]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2230917630704) was called 1 times


In [6]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2230917631856) was called 1 times


In [7]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2230917629696) was called 1 times
Running: Function "load_module_lookup" (id=2230917631424) was called 1 times
Running: Function "load_ID_lookup" (id=2230917630128) was called 1 times
duplicated location faults removed - max duration kept
HOTFIX: Quadrant only faults, PTT Asset Code update


In [8]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i] = fa_floor['Time Left'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:289: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fa_floor['Duration'][i+1] = fa_floor['Duration'][i+1] - fa_floor['Duration'][i]
C:\Users\Jamie.williams\Anaconda3\lib\site-packages\scsavailability\features.py:291: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

In [9]:
df,fa_PTT = feat.create_PTT_df(fa_floor,at,av)

Running: Function "load_PTT_lookup" (id=2230917629840) was called 1 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2230917629552) was called 1 times
Running: Function "load_PTT_lookup" (id=2230917629840) was called 2 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2230917629552) was called 2 times
Running: Function "load_PTT_lookup" (id=2230917629840) was called 3 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2230917629552) was called 3 times
Running: Function "load_PTT_lookup" (id=2230917629840) was called 4 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2230917629552) was called 4 times
Running: Function "load_PTT_lookup" (id=2230917629840) was called 5 times
Previous Hours Weighted
Running: Function "aggregate_availability" (id=2230917629552) was called 5 times
Running: Function "load_PTT_lookup" (id=2230917629840) was called 6 times
Previous Hours Weighted
Running

In [10]:
X,y = md.gen_feat_var(df,target = 'Downtime', features = ['Totes','Faults'])
y=1-y

In [11]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.2,
                                                                 'random_state': None})


R2_cv,R2_OOS,Coeff = md.run_OLS(X_train = X_train,y_train = y_train,X_test = X_test,y_test=y_test, n = 30)

#Problem with this function!




Cross Validation Scores: 
 
        R2 Scores
1       0.504978
2       0.486725
3       0.470600
4       0.143887
5      -3.253572
Mean   -0.329476
STD     1.468130
322
                                 OLS Regression Results                                
Dep. Variable:               Downtime   R-squared (uncentered):                   0.958
Model:                            OLS   Adj. R-squared (uncentered):              0.957
Method:                 Least Squares   F-statistic:                              953.9
Date:                Mon, 04 Jan 2021   Prob (F-statistic):                        0.00
Time:                        13:00:41   Log-Likelihood:                          7848.4
No. Observations:               13599   AIC:                                 -1.506e+04
Df Residuals:                   13281   BIC:                                 -1.267e+04
Df Model:                         318                                                  
Covariance Type:            nonrobust 

### Add LM - metrics

In [12]:
X_train_rd = X_train
Remove = 30

def find_features(X_train, y_train, X):
    max_p = 1
    while max_p > 0.1:
        model = sm.OLS(y_train, X_train)
        results = model.fit()
        top_X = results.pvalues.sort_values(ascending=False).head(X)
        max_p = top_X.tail(1).values[0]
        rm_col = list(results.pvalues.sort_values(ascending=False).head(X).index)
        X_train = X_train.drop(rm_col, axis=1)
    return X_train.columns

In [13]:
fa_Both = fa[fa['Tote Colour']=="Both"]
fa_Both = fa_Both[fa_Both['Pick Station']==False]
fa_Both[fa_Both['Desk']!="Z"]['Asset Code'].value_counts()

C0904STA206    7
Name: Asset Code, dtype: int64

In [14]:
fa[fa['Fault ID']==7647]

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
165705,441848.0,SCS M04: single module 2 access door open fron...,2020-11-25 05:28:29,SCSM04,SCSM04,17.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165738,441848.0,SCS M04: single module 2 access door open fron...,2020-12-04 21:08:53,SCSM04,SCSM04,18.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165720,441848.0,SCS M04: single module 2 access door open fron...,2020-12-02 14:26:23,SCSM04,SCSM04,20.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165721,441848.0,SCS M04: single module 2 access door open fron...,2020-12-02 13:03:40,SCSM04,SCSM04,20.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165740,441848.0,SCS M04: single module 2 access door open fron...,2020-12-04 17:49:05,SCSM04,SCSM04,21.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165722,441848.0,SCS M04: single module 2 access door open fron...,2020-12-02 12:36:34,SCSM04,SCSM04,22.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165715,441848.0,SCS M04: single module 2 access door open fron...,2020-11-29 22:39:41,SCSM04,SCSM04,22.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165730,441848.0,SCS M04: single module 2 access door open fron...,2020-12-06 21:49:15,SCSM04,SCSM04,23.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165718,441848.0,SCS M04: single module 2 access door open fron...,2020-12-03 08:02:11,SCSM04,SCSM04,24.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault
165737,441848.0,SCS M04: single module 2 access door open fron...,2020-12-05 04:45:00,SCSM04,SCSM04,26.0,7647.0,SCSM04,Grey,carousel,SCSM04,False,0.0,1.0,4,Fault


In [15]:
fa[fa['Pick Station']!=False]['Asset Code'].value_counts()

C08PTT082    2458
C12PTT151    1610
C14PTT202    1566
C14PTT201    1422
C09PTT092    1358
C09PTT102    1306
C06PTT031    1110
C12PTT141     989
C09PTT091     963
C08PTT072     913
C09PTT101     894
C08PTT081     884
C13PTT181     866
C13PTT172     863
C05PTT012     840
C12PTT142     817
C14PTT192     813
C13PTT182     791
C14PTT191     777
C11PTT131     759
C13PTT171     749
C11PTT122     726
C06PTT032     718
C11PTT132     698
C07PTT041     672
C08PTT071     669
C07PTT051     661
C06PTT022     579
C12PTT152     565
C07PTT042     562
C05PTT011     528
C06PTT021     497
C10PTT112     457
C10PTT111     430
C07PTT052     410
C11PTT121     390
Name: Asset Code, dtype: int64

In [16]:
pick_stations = ['PTT011','PTT012','PTT021','PTT022','PTT031','PTT032','PTT041','PTT042','PTT051','PTT052','PTT071','PTT072','PTT081','PTT082','PTT091','PTT092','PTT101','PTT102','PTT111','PTT112','PTT121','PTT122','PTT131','PTT132','PTT141','PTT142','PTT151','PTT152','PTT171','PTT172','PTT181','PTT182','PTT191','PTT192','PTT201','PTT202']

In [17]:
fa_floor[fa_floor['Duration']==0]

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type


In [18]:
unmapped

,Asset,Occurrence
0,C08,26
1,C06,9
2,C0904STA206,7
3,C14,4
4,C09,4
5,C13,4
6,C05,3
7,C11,3
8,C10,3
9,C12,1


In [19]:
fa[fa['Asset Code'] == 'C08']

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
168514,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:32,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168515,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:28,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168513,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:34,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168512,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:36,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168511,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:54,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168525,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:08:57,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168527,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:08:42,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168526,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:08:56,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168528,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:08:40,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault
168517,71170.0,C08: C08SCSP07+S13 emergency stop button actua...,2020-11-28 04:09:22,C08,Z,1.0,3436.0,C08,Both,NaN,NaN,False,8.0,2.0,C08 External,Fault


In [20]:
np.exp(X/100)-1

,C05,C0501ACH011,C0501RDC004,C0501STA002,C0501STA012,C0501STA020,C0501STA201,C0501STA217,C0501STA219,C0501STA228,...,SCS202,SCS203,SCS204,SCS205,SCS206,SCSM20,C1404RDC146,C1404STA149,C14PTT202,TOTES
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000,0.436491
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000,0.384723
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000,0.266596
3,0.0,0.0,0.0,0.0,0.072508,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000,0.398406
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000,0.390041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,29.569415,4.365556,0.000000,7.124044e+01,0.0,0.447735,0.0,0.000000,0.000000,0.636675
470,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,20.115344,16.814273,1.104419e+08,0.0,2.596640,0.0,0.000000,0.000000,0.808805
471,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,316.348329,0.000000,7.004469,6.108268e+04,0.0,0.000000,0.0,89.921819,1.974274,0.710296
472,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,234.097424,5.233887e+00,0.0,0.000000,0.0,0.000000,0.051271,0.688206


In [22]:
Coeff

,index,Coefficient
0,C0501STA230,-0.000199
1,C0502STA030,-0.000304
2,C0502STA072,-0.000402
3,C0503ACH234,-0.000231
4,C0503STA226,-0.009339
...,...,...
193,C1404STA130,-0.000212
194,C1404STA177,-0.000155
195,C1404STA181,-0.001419
196,C1404STA197,-0.000273


In [25]:
fa_floor

,Number,Alert,timestamp,PLC,Desk,Duration,Fault ID,Asset Code,Tote Colour,Loop,Suffix,Pick Station,PLCN,Quadrant,MODULE,Alert Type
0,420042.0,SCS122: sto: timeout bin from lifter to car X:...,2020-11-20 00:00:00,SCSM12,SCS122,47.0,5105.0,SCS122,Grey,carousel,SCSM12,False,0.0,3.0,12,Fault
1,186939.0,C23: C2301RDC061 061AQ1 roller motor unit fault,2020-11-20 00:00:00,C23,P01,1.0,3429.0,C2301RDC061,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault
2,186939.0,C23: C2301RDC061 061AQ1 roller motor unit fault,2020-11-20 00:00:00,C23,P01,1.0,3429.0,C2301RDC061,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault
3,99383.0,C12: C1202STA077 077B14 clearance sensor fault,2020-11-20 00:00:00,C12,P02,53.0,3419.0,C1202STA077,Grey,Module,C1202,False,12.0,3.0,14,Fault
4,186939.0,C23: C2301RDC061 061AQ1 roller motor unit fault,2020-11-20 00:00:00,C23,P01,1.0,3429.0,C2301RDC061,Blue,Outside,C2301,False,23.0,0.0,NaN,Fault
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145130,67162.0,C08: C0802STA066 SCS runtime fault,2020-12-10 00:00:00,C08,P02,529.0,3424.0,C0802STA066,Grey,Module,C0802,False,8.0,2.0,7,Fault
145131,188345.0,C23: C2303BCL151 151B54 sensor fault,2020-12-10 01:00:00,C23,P03,3293.0,3428.0,C2303BCL151,Blue,Outside,C2303,False,23.0,0.0,NaN,Fault
145132,130631.0,C16: C1602ACL040 040QM11 motor circuit breaker...,2020-12-10 01:00:00,C16,P02,3600.0,3430.0,C1602ACL040,Blue,NaN,NaN,False,16.0,0.0,NaN,Fault
145133,130631.0,C16: C1602ACL040 040QM11 motor circuit breaker...,2020-12-10 02:00:00,C16,P02,3600.0,3430.0,C1602ACL040,Blue,NaN,NaN,False,16.0,0.0,NaN,Fault
